<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Analyzing%20audio/Speech_to_speech_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sounddevice
!pip install scipy
!pip install playsound
!apt-get install -y libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

!apt install ffmpeg
!apt-get install libportaudio2

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mc155ida
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mc155ida
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import argparse
import openai
import pandas as pd
import time
from IPython.display import Markdown, display, Image, Audio
from google.colab import files
import base64
import requests
import shutil
import scipy.io.wavfile
import sounddevice
import sqlite3
import time
import os
import numpy as np
import re

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

from google.colab import drive, userdata,  output
from openai import OpenAI
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    API_KEY = userdata.get('OPENAI_API_KEY')
    if not API_KEY:
        raise ValueError("API key not found. Please add it to Colab Secrets.")
except ImportError:
    # Fallback for environments other than Colab, though not recommended
    API_KEY = os.environ.get("OPENAI_API_KEY")
    if not API_KEY:
        raise ValueError("API_KEY environment variable not set.")

API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key= API_KEY)

In [ ]:
def transcribe(audio_path):
    """ Transcribe audio file to text.

    Args:
        audio_path: path to audio file.

    Returns:
        transcribed text.
    """
    with open(audio_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file, model='whisper-1')
        return transcription.text


In [ ]:
def create_prompt(to_translate, to_language):
    """ Generate prompt to translate text to target language.

    Args:
        to_translate: translate this text.
        to_language: translate text to this language.

    Returns:
        Translated text.
    """
    parts = []
    parts += [f'Translate this text to {to_language}:']
    parts += [to_translate]
    parts += ['Translated text:']
    return '\n'.join(parts)

In [ ]:
def call_llm(prompt):
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {'role':'user', 'content':prompt}
                    ]
                )
            return response.choices[0].message.content
        except:
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query OpenAI model!')


In [ ]:
def generate_speech(speech_text):
    """ Generates speech for given text.

    Args:
        speech_text: generate speech for this text.

    Returns:
        query result.
    """
    response = client.audio.speech.create(
        model='tts-1', voice='alloy',
        input=speech_text)
    return response.content

In [ ]:
"""
Hallo Leute, herzlich willkommen, heute sprechen wir über LLMOPS. Bei diesem Thema geht es um die Zukunft der Mensch-KI-Zusammenarbeit.

"""

from google.colab import output
output.enable_custom_widget_manager()

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
"""
The above recording goes to the Data/audio_data/translate_german_to_french.wav

"""


file_name = 'translate_german_to_french.wav'

drive_directory = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/'
drive_path = os.path.join(drive_directory, file_name)

print(f"Ensuring directory exists: {drive_directory}")
os.makedirs(drive_directory, exist_ok=True)

with open('recording.webm', 'wb') as f:
  f.write(recorder.audio.value)

print(f"Converting to .wav format: {file_name}")
!ffmpeg -i recording.webm -ac 1 -f wav {file_name} -y -hide_banner -loglevel panic

print(f"Copying file to Google Drive: {drive_path}")
!cp {file_name} "{drive_path}"

os.remove('recording.webm')
os.remove(file_name)

print("\n✅ Recording successfully saved to your Google Drive!")
print(f"File path: {drive_path}")

Ensuring directory exists: /content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/
Converting to .wav format: translate_german_to_french.wav
Copying file to Google Drive: /content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/translate_german_to_french.wav

✅ Recording successfully saved to your Google Drive!
File path: /content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/translate_german_to_french.wav


In [ ]:
audio_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/translate_german_to_french.wav'



to_translate = transcribe(audio_path)
print(f'Original text: {to_translate}')

prompt = create_prompt(to_translate, "French")
translated = call_llm(prompt)
print(f'Translated text: {translated}')

speech = generate_speech(translated)

print("Playing audio directly from memory...")
display(Audio(data=speech, autoplay=True))



Original text: Hallo Leute, herzlich willkommen. Heute sprechen wir über LLM Ops. Bei diesem Thema geht es um die Zukunft der Menschen, die zusammenarbeiten.
Translated text: Bonjour tout le monde, bienvenue. Aujourd'hui, nous parlons des opérations LLM. Ce sujet concerne l'avenir des personnes qui collaborent.
Playing audio directly from memory...
